In [10]:
import pandas as pd
import requests
import settings

In [15]:
headers = {"User-Agent": settings.email_address}
ticker = "TSLA"

In [6]:
def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "_")
    ticker_json = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers).json()
    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f'Ticker: {ticker} not found".format(ticker=ticker)')



In [7]:
cik = cik_matching_ticker('tsla')
print(cik)

0001318605


In [21]:
def get_submission_data_for_ticker(ticker, headers=headers, only_fillings_df=False):
    cik = cik_matching_ticker(ticker, headers=headers)
    headers = headers.copy()
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    if only_fillings_df:
        return pd.DataFrame(company_json["filings"]["recent"])
    else:
        return company_json


In [22]:
data = get_submission_data_for_ticker(ticker, only_fillings_df=True)

In [23]:
data.keys()

Index(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime',
       'act', 'form', 'fileNumber', 'filmNumber', 'items', 'core_type', 'size',
       'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'],
      dtype='object')

In [25]:
def get_filtered_filings(ticker, form_type='10-K', just_accession_numbers=False, headers=headers):
    company_filings_df = get_submission_data_for_ticker(ticker, only_fillings_df=True, headers=headers)
    if form_type is not None:
        df = company_filings_df[company_filings_df["form"] == form_type]
        if just_accession_numbers:
            accession_df = df.set_index('reportDate',inplace=True)['accessionNumber']
            return accession_df
        else:
            return df
    else:
        raise ValueError("Must provide form_type")




In [26]:
filings = get_filtered_filings(ticker, form_type='10-K')

In [27]:
filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
56,0001628280-25-003063,2025-01-30,2024-12-31,2025-01-30T01:42:33.000Z,34,10-K,001-34756,25570807,,XBRL,15788647,1,1,tsla-20241231.htm,10-K
170,0001628280-24-002390,2024-01-29,2023-12-31,2024-01-27T02:00:20.000Z,34,10-K,001-34756,24569853,,XBRL,15527801,1,1,tsla-20231231.htm,10-K
271,0000950170-23-001409,2023-01-31,2022-12-31,2023-01-31T02:29:15.000Z,34,10-K,001-34756,23570030,,XBRL,31445171,1,1,tsla-20221231.htm,10-K
371,0000950170-22-000796,2022-02-07,2021-12-31,2022-02-05T01:11:27.000Z,34,10-K,001-34756,22595227,,XBRL,29316024,1,1,tsla-20211231.htm,10-K
503,0001564590-21-004599,2021-02-08,2020-12-31,2021-02-08T12:27:23.000Z,34,10-K,001-34756,21598537,,XBRL,32860345,1,1,tsla-10k_20201231.htm,10-K
641,0001564590-20-004475,2020-02-13,2019-12-31,2020-02-13T12:12:18.000Z,34,10-K,001-34756,20606921,,XBRL,29961626,1,1,tsla-10k_20191231.htm,10-K
794,0001564590-19-003165,2019-02-19,2018-12-31,2019-02-19T11:10:16.000Z,34,10-K,001-34756,19613254,,10-K,30826751,1,0,tsla-10k_20181231.htm,10-K
901,0001564590-18-002956,2018-02-23,2017-12-31,2018-02-23T11:07:43.000Z,34,10-K,001-34756,18634585,,10-K,25498533,1,0,tsla-10k_20171231.htm,10-K
